# Fluid Structure Interaction with SpatialPy

## Definition of the model

### Imports and definitions

In [1]:
import os
import sys
sys.path.insert(1, "../..")
import math
import numpy as np
import matplotlib.pyplot as plt

import spatialpy

In [2]:
class Density(spatialpy.BoundaryCondition):
    def __init__(self, rhoF, rhoB):
        self.rhoF = rhoF
        self.rhoB = rhoB
        
    def expression(self):
        bcstr = "if(me->type == 1 || me->type == 3){"
        bcstr += f"me->rho = {self.rhoF};"
        bcstr += "}"
        bcstr += "if(me->type == 2){"
        bcstr += f"me->rho = {self.rhoB};"
        bcstr += "}"
        return bcstr

In [3]:
class Teleport(spatialpy.BoundaryCondition):
    def __init__(self, nu):
        self.nu = nu
        
    def expression(self):
        bcstr = "if(me->x[0] > system->xhi || me->x[0] < system->xlo){"
        bcstr += "me->x[0] = system->xlo + 1e-6;"
        bcstr += "me->v[0] = 0.0333;"
        bcstr += f"me->nu = {self.nu};"
        bcstr += "}"
        bcstr += "if(me->x[1] > system->yhi){"
        bcstr += "me->v[1]= 0.0;"
        bcstr += "me->x[1]= 99e-6;"
        bcstr += "}"
        bcstr += "if(me->x[1] < system->ylo){"
        bcstr += "me->v[1]= 0.0;"
        bcstr += "me->x[1]= 1e-6;"
        bcstr += "}"
        bcstr += "me->x[2] = 0;"
        return bcstr

### Model

In [4]:
class FluidStructureInteraction(spatialpy.Model):
    def __init__(self, model_name="Fluid Structure Interaction", Ra=1e4):
        spatialpy.Model.__init__(self, model_name)
        
        # System Constants
        nxF, nyF = 60, 100
        nW = 3
        lx, ly = 300e-6, 100e-6
        lbz = -50e-6
        rhoF0, rhoB0 = 1000, 7850
        nu = 1e-3
        vp = 0.33
        E = 2e5
        cF0 = vp
        cB0 = math.sqrt(E/(3*(1-2*vp))/rhoB0)
        
        # Compute Domain Bounds
        dW = dF = ly/nxF
        dB = 0.6*dF
        xMinW, xMaxW = lbz, lx
        yMinTW, yMaxTW = ly, ly+(nW-1)*dW
        yMinBW, yMaxBW = 0-(nW-1)*dW, 0
        xMinB, xMaxB = 100e-6, 105e-6
        yMinB, yMaxB = yMinBW, 50e-6
        
        # Compute Volume and Mass per Particle
        vol = (lx-lbz)*(ly+2*nW*dW)
        volW = 2*(nW-1)*dW*(lx-lbz)
        volB = 5e-6*50e-6
        volF = vol - volW - volB
        mPFP = mPWP = volF*rhoF0/(nxF*nyF)
        mPBP = volB*rhoB0/500
        
        # Domain
        self.staticDomain = False
        self.domain = spatialpy.Domain(0, xlim=(lbz, lx), ylim=(yMinBW, yMaxTW), zlim=(0,0), gravity=[0, -1, 0])
        for x_ndx, x in enumerate(np.arange(lbz, lx+dF, dF)):
            for y_ndx, y in enumerate(np.arange(yMinBW, yMaxTW+dF, dF)):
                # Top and Bottom Walls
                if y > ly-1e-6 or y <= 0:
                    self.domain.add_point([x, y, 0], type=3, vol=volF/(nxF*nyF), mass=mPWP, nu=nu, fixed=True)
                # Fluid
                elif x < 0:
                    self.domain.add_point([x, y, 0], type=1, vol=volF/(nxF*nyF), mass=mPFP, nu=nu, fixed=False)
                    
        for x_ndx, x in enumerate(np.arange(xMinB, xMaxB+dB, dB)):
            for y_ndx, y in enumerate(np.arange(yMinB, yMaxB+dB, dB)):
                # Beam
                self.domain.add_point([x, y, 0], type=2, vol=volB/500, mass=mPBP, nu=nu, fixed=False)
                
        # Boundary Conditions
        self.add_boundary_condition(Density(rhoF0, rhoB0))
        self.add_boundary_condition(Teleport(nu))
                
        # Timespan
        #self.timespan(np.arange(0, 10+1e-8, 1e-4), timestep_size=1e-8)
        self.timespan(np.arange(0, 10+1e-4, 1e-2), timestep_size=1e-4)

In [5]:
model = FluidStructureInteraction()

In [6]:
model.domain.dimensions = 2
model.domain.plot_types()

## Running model and processing the results

In [7]:
from spatialpy import Solver
solver = Solver(model=model, debug_level=0)
%time solver.compile()
print(solver.build_dir)

CPU times: user 550 ms, sys: 327 µs, total: 550 ms
Wall time: 3.26 s
/tmp/spatialpy_build_d68u9rh2


In [8]:
%time results = solver.run()

Terminated by user after seconds: 3981.90
CPU times: user 98.4 ms, sys: 18.2 ms, total: 117 ms
Wall time: 1h 6min 21s


In [17]:
results.plot_property("type",t_ndx=0)

In [12]:
results.plot_property("type",t_ndx=1)

In [13]:
#results.plot_property("type", animated=True)

In [ ]:
points, properties = results.read_step(2)
for i, point in enumerate(points):
    if properties['type'][i] == 1:
        print(point)

In [ ]:
print(model.domain.xlim)

In [ ]:
print(model.domain.ylim)